# Modelo de Recomendaciones:

El modelo de recomendaicones personaliadas puede basarse en diferentes enfoques.
Entonces una viabilidad más sencilla y efectiva es el usar "algoritmos de clustering" (agrupamiento), como K-means, esto con el fin de identificar patrones en los datos financieros de los usuarios y hgacer recomendaciones basados en esos parones.

Como objetivo:
- Agrupar los usuarios según sus patornes de gastos.
- Proporcionar reocmenaicones personaliadas para reduir gastos en categorías donde están gastando mpas de lo reocmendado.

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import joblib

# Simulación de datos de gasto por categorías para 10 usuarios, permitiendo ceros
np.random.seed(42)
usuarios = np.arange(1, 11)
alimentacion = np.random.uniform(0, 800, size=10)  # Permitir valores de 0
transporte = np.random.uniform(0, 500, size=10)    # Permitir valores de 0
entretenimiento = np.random.uniform(0, 300, size=10) # Permitir valores de 0
vivienda = np.random.uniform(0, 1200, size=10)     # Permitir valores de 0

# Crear un DataFrame con los datos simulados
data = pd.DataFrame({
    'Usuario': usuarios,
    'Alimentación': alimentacion,
    'Transporte': transporte,
    'Entretenimiento': entretenimiento,
    'Vivienda': vivienda
})


# Mostrar los datos simulados
data


,Usuario,Alimentación,Transporte,Entretenimiento,Vivienda
0,1,299.632095,10.292247,183.555868,729.053822
1,2,760.571445,484.954926,41.848158,204.628948
2,3,585.595153,416.221320,87.643395,78.061912
3,4,478.926787,106.169555,109.908553,1138.662645
4,5,124.814912,90.912484,136.820995,1158.758440
5,6,124.795616,91.702255,235.552788,970.076818
6,7,46.466890,152.121121,59.902135,365.536523
7,8,692.940917,262.378216,154.270332,117.206537
8,9,480.892009,215.972509,177.724371,821.079632
9,10,566.458062,145.614570,13.935124,528.182992


Entrenamos el modelo de Clustering (K-Means) para agrupar a los usuarios según sus hábitos de gasto en diferentes categorías.
Primero instalamos 'scikit-learn'


Entrenamos el modelo de K-means

In [2]:

# Entrenar el modelo K-Means con 3 clusters
X = data[['Alimentación', 'Transporte', 'Entretenimiento', 'Vivienda']]
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)

KMeans(n_clusters=3, random_state=42)

In [21]:
# Guardar el modelo entrenado en un archivo
joblib.dump(kmeans, 'modelo_recomendaciones.pkl')

print("Modelo K-Means entrenado y guardado en 'modelo_recomendaciones.pkl'.")

Modelo K-Means entrenado y guardado en 'modelo_recomendaciones.pkl'.


### Agregamos el modelo K-means a FastAPI

In [12]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import joblib
import numpy as np

In [13]:
# Crear la aplicación FastAPI
app = FastAPI()

In [14]:
# Cargar los modelos ARIMA y K-Means entrenados
modelo_arima = joblib.load('modelo_arima.pkl')
# Cargar el modelo K-Means entrenado
try:
    kmeans = joblib.load('modelo_recomendaciones.pkl')
except Exception as e:
    print(f"Error cargando el modelo K-Means: {e}")
    kmeans = None


Error cargando el modelo K-Means: [Errno 2] No such file or directory: 'modelo_recomendaciones.pkl'


In [15]:
# Modelo de entrada para la predicción de ARIMA
class PrediccionEntrada(BaseModel):
    pasos: int  # Número de pasos (meses) para predecir a futuro

In [16]:
# Modelo de entrada para K-Means (gasto por categorías)
class GastoEntrada(BaseModel):
    alimentacion: float
    transporte: float
    entretenimiento: float
    vivienda: float

In [17]:
# Endpoint para predicción de gastos futuros con ARIMA
@app.post("/predecir-gastos/")
def predecir_gastos(datos: PrediccionEntrada):
    try:
        # Realizar la predicción para los próximos N pasos (meses)
        prediccion = modelo_arima.forecast(steps=datos.pasos)
        return {"gastos_futuros": list(prediccion)}
    
    except Exception as e:
        # Capturar cualquier excepción y devolverla en la respuesta
        return {"error": str(e)}

In [18]:
# Endpoint para recomendaciones personalizadas con K-Means
@app.post("/recomendar/")
def recomendar(datos: GastoEntrada):
    try:
        # Crear un DataFrame con los nombres de columnas que el modelo K-Means espera
        entrada = pd.DataFrame([[
            datos.alimentacion,
            datos.transporte,
            datos.entretenimiento,
            datos.vivienda
        ]], columns=['Alimentación', 'Transporte', 'Entretenimiento', 'Vivienda'])

        # Hacer la predicción con K-Means
        cluster = kmeans.predict(entrada)[0]

        # Generar recomendaciones basadas en el cluster
        recomendaciones = {
            0: "Considera reducir tus gastos en alimentación y entretenimiento.",
            1: "Transporte está representando una gran parte de tus gastos. Prueba alternativas más económicas.",
            2: "Tus gastos en vivienda son altos. Considera opciones de ahorro en este rubro."
        }

        return {"recomendacion": recomendaciones.get(cluster, "Recomendación no disponible.")}
    
    except Exception as e:
        # Capturar cualquier excepción y devolverla en la respuesta
        return {"error": str(e)}

In [20]:
if __name__ == "__main__":
    import nest_asyncio
    import uvicorn
    nest_asyncio.apply()  # Esto es necesario si estás en Jupyter
    uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [17712]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [17712]
